In [ ]:
import requests
import pandas as pd
import time
from dotenv import load_dotenv
import os

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Retrieve the API key
API_KEY = os.getenv("GOOGLE_API_KEY")

In [ ]:
# Lê o ficheiro que já criaste
df_locais = pd.read_csv("google_places_AMP.csv")

In [12]:
# Filtrar apenas os municípios da Área Metropolitana do Porto
municipios_amp = ['Arouca', 'Espinho', 'Gondomar', 'Maia', 'Matosinhos', 'Oliveira de Azeméis', 'Paredes', 'Penafiel', 'Porto', 'Póvoa de Varzim', 'Santa Maria da Feira', 'Santo Tirso', 'São João da Madeira', 'Trofa', 'Valongo', 'Vila do Conde', 'Vila Nova de Gaia']
df_locais = df_locais[df_locais["Cidade"].isin(municipios_amp)].reset_index(drop=True)
print(f"Total de locais nos municípios da AMP: {len(df_locais)}")

Total de locais nos municípios da AMP: 1890


In [13]:
# Lista onde vamos guardar as reviews
todas_reviews = []

In [14]:
# Função para obter place_id de um local
def obter_place_id(nome_local, cidade):
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {
        "query": f"{nome_local}, {cidade}, Portugal",
        "key": API_KEY
    }
    resposta = requests.get(url, params=params)
    resultados = resposta.json().get("results")
    if resultados:
        return resultados[0].get("place_id")
    return None

In [15]:
# Função para obter reviews com base no place_id
def obter_reviews(place_id):
    url = "https://maps.googleapis.com/maps/api/place/details/json"
    params = {
        "place_id": place_id,
        "fields": "review,name",
        "key": API_KEY
    }
    resposta = requests.get(url, params=params)
    return resposta.json().get("result", {}).get("reviews", [])

In [16]:
# Loop por cada local
for index, row in df_locais.iterrows():
    nome = row["Nome"]
    cidade = row["Cidade"]
    categoria = row["Categoria"]

    print(f"📍 A procurar comentários para: {nome} ({cidade})")
    place_id = obter_place_id(nome, cidade)
    time.sleep(1.5)  # Espera para evitar sobrecarga

    if place_id:
        reviews = obter_reviews(place_id)
        for review in reviews:
            autor = review.get("author_name")
            texto = review.get("text")
            data = review.get("relative_time_description")
            rating = review.get("rating")
            todas_reviews.append([cidade, categoria, nome, autor, texto, data, rating])
    else:
        print(f"⚠️ Place ID não encontrado para: {nome}")

📍 A procurar comentários para: Tasquinha da Quinta (Arouca)


📍 A procurar comentários para: Parlamento (Arouca)
📍 A procurar comentários para: Casa Testinha (Arouca)
📍 A procurar comentários para: A Assembleia (Arouca)
📍 A procurar comentários para: Pedestre 142 (Arouca)
📍 A procurar comentários para: Tasquinho do Parque (Arouca)
📍 A procurar comentários para: Varandinha (Arouca)
📍 A procurar comentários para: Alto da Estrada (Arouca)
📍 A procurar comentários para: Restaurante do Pedrogão (Arouca)
📍 A procurar comentários para: Paiva à Vista (Arouca)
📍 A procurar comentários para: Quinta D'Além da Ponte (Arouca)
📍 A procurar comentários para: Casa no Campo (Arouca)
📍 A procurar comentários para: Pão de Ló de Arouca (Arouca)
📍 A procurar comentários para: Café Arouquense (Arouca)
📍 A procurar comentários para: Abadessa Restaurante e Petiscos (Arouca)
📍 A procurar comentários para: O Júlio (Arouca)
📍 A procurar comentários para: Churrascaria Central (Arouca)
📍 A procurar comentários para: O Mineiro de Regoufe (Arouca)
📍 A procurar comentários para

In [17]:
# Criar DataFrame final
df_reviews = pd.DataFrame(todas_reviews, columns=[
    "Cidade", "Categoria", "Nome_Local", "Autor", "Texto", "Data", "Rating"
])

In [ ]:
# Guardar em CSV
df_reviews.to_csv("comments_google_maps_AMP.csv", index=False, encoding='utf-8-sig')

print(f"\n✅ Recolhidas {len(df_reviews)} reviews! Guardado em 'comments_google_maps_AMP.csv'")


✅ Recolhidas 9212 reviews! Guardado em 'comentarios_google_maps_AMP_2.csv'
